In [ ]:
# %run /home/ptenkaate/scratch/Master-Thesis/convert_ipynb_to_py_files.ipynb

In [1]:
import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Resize, Compose, ToTensor, Normalize

import argparse
import os
import math 
import skimage
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

import time
import pickle

from datetime import datetime
from pathlib import Path

from py_files.new_dataset import *

from py_files.cnn_model import *
from py_files.pigan_model import *

from py_files.seq_pi_gan_functions import *

# %matplotlib qt

C:\Users\pault\anaconda3\lib\site-packages\kornia\augmentation\augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


Imported CNN and Mapping functions.
Imported PI-Gan model.
Loaded all helper functions.


In [ ]:
plt.plot([0, 1])

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Import classes

In [2]:
# DEVICE = set_device()
DEVICE = torch.device("cpu")

print('----------------------------------')
print('Using device for training:', DEVICE)
print('----------------------------------')

----------------------------------
Using device for training: cpu
----------------------------------


C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Set the run you want to evaluate 

In [3]:
sorted(os.listdir(path='saved_runs'))

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['pi-gan 2021-06-11 17_41_08_',
 'pi-gan 2021-06-12 07_53_55_',
 'pi-gan 2021-06-13 08_16_37_',
 'pi-gan 2021-06-13 08_17_54_',
 'pi-gan 2021-06-13 08_24_54_',
 'pi-gan 2021-06-13 14_26_23_',
 'pi-gan 2021-06-13 14_26_56_',
 'pi-gan 2021-06-13 14_27_20_',
 'pi-gan 2021-06-14 07_56_37_',
 'pi-gan 2021-06-14 08_00_11_',
 'pi-gan 2021-06-14 08_00_58_',
 'pi-gan 2021-06-14 08_01_58_',
 'pi-gan 2021-06-14 08_02_18_',
 'pi-gan 2021-06-14 08_02_57_',
 'pi-gan 2021-06-14 08_06_22_',
 'pi-gan 2021-06-14 08_18_35_',
 'pi-gan 2021-06-14 09_56_28_',
 'pi-gan 2021-06-14 10_12_35_',
 'pi-gan 2021-06-14 11_46_35_',
 'pi-gan 2021-06-23 12_29_32_',
 'pi-gan 2021-06-23 12_30_50_',
 'pi-gan 2021-06-23 12_40_11_',
 'pi-gan 2021-06-23 12_42_57_']

In [4]:
run = sorted(os.listdir(path='saved_runs'))[-1]
# run = "pi-gan 2021-06-05 00:58:36 "

ARGS = load_args(run, print_changed=False)

print(run)

for var, val in vars(ARGS).items():
    print(f"{var.ljust(15)} \t {val}")

pi-gan 2021-06-23 12_42_57_
device          	 GPU
print_models    	 False
name            	 
pretrained      	 None
pretrained_best_dataset 	 train
pretrained_best_loss 	 mask
pretrained_models 	 None
pretrained_lr_reset 	 None
dataset         	 new
seed            	 34
rotate          	 False
translate       	 True
translate_max_pixels 	 20
flip            	 False
crop            	 False
stretch         	 True
stretch_factor  	 1.2
norm_min_max    	 [0, 1]
pcmra_epochs    	 0
mask_epochs     	 5000
batch_size      	 24
eval_every      	 50
shuffle         	 True
n_coords_sample 	 5000
min_lr          	 1e-05
cnn_setup       	 -1
pcmra_train_cnn 	 True
mask_train_cnn  	 True
mapping_setup   	 -1
dim_hidden      	 256
siren_hidden_layers 	 3
first_omega_0   	 30.0
hidden_omega_0  	 30.0
pcmra_first_omega_0 	 30.0
pcmra_hidden_omega_0 	 30.0
cnn_lr          	 0.0001
cnn_wd          	 0
mapping_lr      	 0.0001
pcmra_mapping_lr 	 0.0001
siren_lr        	 0.0001
siren_wd        	 0
pcmra_s

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Evaluation

In [5]:
ARGS.batch_size = 2

train_dl, val_dl, test_dl = initialize_dataloaders(ARGS)

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


----------------------------------
Using device for training: cpu
----------------------------------
Train subjects: 86
Val subjects: 29
Test subjects: 29


#### Load models

In [6]:
ARGS.print_models=False
models, optims, schedulers = load_models_and_optims(ARGS)

----------------------------------
Using device for training: cpu
----------------------------------


C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CNN
MAPPING
SIREN
PCMRA_MAPPING
PCMRA_SIREN


In [7]:
# models["pcmra_siren"]

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
ARGS.pretrained_best_dataset = "train"
ARGS.pretrained_best_loss = "mask"

load_pretrained_models(run, ARGS.pretrained_best_dataset, ARGS.pretrained_best_loss,
                       models, optims, DEVICE, pretrained_models=None)

Loading params from cnn


C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading params from mapping
Loading params from siren
Loading params from pcmra_mapping
Loading params from pcmra_siren


In [9]:
##### loss function #####
mask_criterion = nn.BCELoss()    
pcmra_criterion = nn.MSELoss()  
dice_criterion = calc_dice_loss

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def scroll_through_output(dataloader, shape=[24, 64, 64], transform=False):
    with torch.no_grad():
        pcmras = masks = loss_covers = mask_outs = pcmra_outs = torch.Tensor([])

        titles = []

        mask_losses, pcmra_losses, dice_losses = [], [], []

        blur_layer = initialize_blurring_layer(1.0, DEVICE)
    
        for batch in dataloader: 

            if transform:
                batch = transform_batch(batch, ARGS)


            _, _, _, pcmra, coords_array, pcmra_array, mask_array, surface_array, norm_array = \
                get_siren_batch(batch, blur_layer, ARGS)


            mask_out = get_complete_image(models, pcmra, coords_array, ARGS)
            pcmra_out = get_complete_image(models, pcmra, coords_array, ARGS, output="pcmra")

            mask_loss = mask_criterion(mask_out, mask_array) 
            dice_loss = dice_criterion(mask_out, mask_array) 
            pcmra_loss = pcmra_criterion(pcmra_out, pcmra_array) 

            mask_losses.append(mask_loss.item())
            dice_losses.append(dice_loss.item())
            pcmra_losses.append(pcmra_loss.item())

            for p, m, po, mo in zip(pcmra_array, mask_array, pcmra_out, mask_out):
                pcmras = torch.cat((pcmras, p.cpu().view(shape).detach().permute(1, 2, 0)), 2)
                masks = torch.cat((masks, m.cpu().view(shape).detach().permute(1, 2, 0)), 2)
                mask_outs = torch.cat((mask_outs, mo.cpu().view(shape).detach().permute(1, 2, 0)), 2)
                pcmra_outs = torch.cat((pcmra_outs, po.cpu().view(shape).detach().permute(1, 2, 0)), 2)
            
            break
        
        print("Mask loss: ", np.array(mask_losses).mean())
        print("Dice loss: ", np.array(dice_losses).mean())
        
        print("PCMRA loss:", np.array(pcmra_losses).mean())

        titles = "Results"

        window = Show_images(titles, (pcmras.numpy(), "pcmras"), 
                                     (pcmra_outs.numpy(), "pcmra output"),
                                     (masks.numpy(), "masks"),
                                     (mask_outs.numpy(), "mask output"), 
                            )

        return window

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
ARGS.rotate = True
ARGS.translate = True
ARGS.flip = True
ARGS.crop = True
ARGS.stretch = True
ARGS.n_coords_sample = -1

shape = [24, 128, 128]
window = scroll_through_output(train_dl, shape=shape, transform=False)
# window = scroll_through_output(val_dl, shape=shape)
# window = scroll_through_output(test_dl, shape=shape)

C:\Users\pault\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mask loss:  0.01560133695602417
Dice loss:  0.12774401903152466
PCMRA loss: 0.028880443423986435


In [ ]:
def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [ ]:
# criterion = nn.MSELoss()

# for idx, subj, proj, pcmra, coords, pcmra_array, mask_array in val_dl:
#     if idx == 1:
#         break

# print(pcmra.shape)
# out = get_complete_image(models, pcmra, coords, ARGS, output="pcmra")
# out = out.view(1, 1, 64, 64, 24).permute(0, 1, 4, 2, 3)

# plt.imshow(pcmra[0, 0, 8, :, :].cpu())
# plt.show()

# plt.imshow(out[0, 0, 8, :, :].cpu().detach())
# plt.show()

# print(criterion(pcmra, out).item())

In [ ]:
a = torch.randn(24, 1, 24, 128, 128).cuda()
print(a)


In [ ]:
t = time.time()

a = torch.floor(torch.round(a*10 ) / 10)

print(time.time() - t)


In [ ]:
print(a)